# Create the banner for our GitHub repo and docs

This notebook creates the banner for our repos, and also illustrates how to quickly measure location and velocity of a bird in a cage

In [ ]:
import numpy as np
import birdwatcher as bw                   # general functionality
import birdwatcher.movementdetection as md # higer level functionality for movement detection

We use the example video that comes with birdwatcher

In [ ]:
vfs = bw.testvideosmall()

In [ ]:
md.detect_movement?

We apply a movement detection function, based on background subtraction

In [ ]:
# first define parameters, default algorithm is MOG2
settings = {'bgs_params': {'VarThreshold': 200,
                           'NMixtures': 8, 
                           'History': 3},
            'processing': {'morphologyex': 2,
                          'ignore_firstnframes': 20}}

# do movementdetection based on those parameters
coords, coordscount, coordsmean = md.detect_movement(vfs, settings=settings, analysispath='output/', overwrite=True)

`coords` is an array that has the coordinates of the positive pixels (i.e. changed because of moving bird) for each frame

`coordscount` is an array with the total number of positive pixels for each frame

`coordsmean` is an array with the mean of changed pixel coordinates for each frame 

Let's create a video based on the movement pixels. We want an animated gif of positive pixels in orange, cropped, with measurement values superimposed on it. You can look all methods up in the documentation: https://birdwatcher.readthedocs.io/en/latest/api.html#module-birdwatcher.frameprocessing

Note that the code for measuring speed etc is not optimized in any way. It is just to create a short example for the banner. 

In [ ]:
# a list of strings, each representing the number of suprathreshold pixels of the corresponding frame in the video
counttextlist = [f'count : {c:4}' for c in coordscount]

In [ ]:
# an array of the spatial mean of suprathreshold pixels per frame in the video
# frames with count <= 20, which may mostly be random noise, are set to nan
coordsmean = np.array([coord if c > 20 else (np.nan, np.nan) for (coord,c) in zip(coordsmean,coordscount)])
#  turn this into a list of strings that we can display in the video
meantextlist = [f'mean : ({x:.0f}, {y:.0f})' for (x,y) in coordsmean ]

In [ ]:
## create a list of strings indicating the velocity of the bird 
## (which is the difference between spatial mean of adjacent frames)
diff = np.zeros_like(coordsmean)
# difference in distance between adjacent spatial means
diff[1:] = np.diff(coordsmean, axis=0)
# we include for speed only when many pixels change (>400), which is when bird moves around
speed = [(x**2+y**2)*0.5 if c>400 else 0 for ((x,y),c) in zip(diff,coordscount) ]
speedtextlist = [f'velocity : {int(s):4}' for s in speed]

In [ ]:
# create the video; we choose gif as format, which works well for banners 
vfs_gif = (coords.iter_frames(nchannels=3, value=(0,140,255), dtype='uint8')
             .crop(250,450,0,1280)
             .draw_text(counttextlist, org=(1030,25),fontscale=0.8)
             .draw_text(meantextlist, org=(1030,50),fontscale=0.8)
             .draw_text(speedtextlist, org=(1030,75),fontscale=0.8)
             .tovideo('output/banner_vt200_ex2.gif', format='gif', framerate=25, crf=None, codec=None, pixfmt=None))